This .ipynb file was only created to test certain modules, functions to check wether it is working or not and even
how to optimally update the code. 

Main file is app.py.

In [2]:
from pinecone import Pinecone
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

d:\Projects\CallPrep\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
def chunk_text(text, chunk_size=400, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    # document = Document(page_content=text)
    chunked_texts = text_splitter.split_documents(text)
    # return [chunk.page_content for chunk in chunked_texts]
    return chunked_texts

In [4]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [5]:
doc=read_doc('docs/')
doc

[]

In [6]:
embeddings_model = SentenceTransformerEmbeddings(model_name = "all-MiniLm-L6-v2")

# Function to generate embeddings
def generate_embeddings(chunked_texts):
    vector_embeddings = embeddings_model.embed_documents(chunked_texts)
    return vector_embeddings

d:\Projects\CallPrep\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
pc = Pinecone(api_key="8b4a6ab8-9fbb-45ce-b654-b269644b091a")
index_name = "doc-archive"
index = pc.Index(index_name)

In [10]:
query = "water resource"
query_embedding = embeddings_model.embed_query(query)
search_results = index.query(vector=query_embedding, include_metadata=True, top_k=20)

In [11]:
search_results

{'matches': [{'id': 'WWF_CaseStudies_final.pdf_67',
              'metadata': {'filename': 'WWF_CaseStudies_final.pdf'},
              'score': 0.58495909,
              'values': []},
             {'id': 'WWF_CaseStudies_final.pdf_95',
              'metadata': {'filename': 'WWF_CaseStudies_final.pdf'},
              'score': 0.581298411,
              'values': []},
             {'id': 'WWF_CaseStudies_final.pdf_100',
              'metadata': {'filename': 'WWF_CaseStudies_final.pdf'},
              'score': 0.551592469,
              'values': []},
             {'id': 'WWF_CaseStudies_final.pdf_17',
              'metadata': {'filename': 'WWF_CaseStudies_final.pdf'},
              'score': 0.548508167,
              'values': []},
             {'id': 'WWF_CaseStudies_final.pdf_76',
              'metadata': {'filename': 'WWF_CaseStudies_final.pdf'},
              'score': 0.547255397,
              'values': []},
             {'id': 'WWF_CaseStudies_final.pdf_18',
              'met

In [21]:
document = []
for match in search_results.matches:
    filename = match.metadata['filename']
    document.append(filename)

In [22]:
document

['harshibars-resume.pdf',
 'harshibars-resume.pdf',
 'Resume@Dibakar-2.pdf',
 'Resume@Dibakar-2.pdf',
 'harshibars-resume.pdf']

In [ ]:
documents = []
for match in search_results.results[0].matches:
    metadata = match.metadata
    document_name = metadata.get('filename')
    if document_name:
        documents.append({"filename": document_name})